---
# Python 人工智能入门 毕业项目
# 为某平台搭建金融风控模型
## Joe(请写下你的名字)
## 2049年10月1日(请写下你完成报告的日期)
---

## 目录
<ul>
<li><a href="#definition">定义</a></li>
<li><a href="#analysis">分析</a></li>
<li><a href="#implementation">实现</a></li>
<li><a href="#result">结果</a></li>
<li><a href="#conclusion">结论</a></li>   
</ul>



---
<a id="definition"></a>
## Ⅰ.定义
我们从[kaggle](https://www.kaggle.com/wendykan/lending-club-loan-data)获得部分信贷平台的数据，我们需要从这些数据中，挖掘出我们有用的信息，用来给借贷平台粗略的搭建一个模型，来预测在哪些情况下借贷的风险较高。

### 1.项目概况
#### 1.1 项目的背景信息
现在，借贷平台已经很常见，主要功能是给一些需要资金帮助的人群，提供金融服务，来帮助这些人群解除燃眉之急，最典型的如招行的消费贷。但是在有写情况下，会出现借贷人无法支付本息，更有甚者，连本金都无法支付，就会对金融平台产生坏账，这是需要我们解决的问题。
#### 1.2 做这个项目的出发点
上述所说的问题，我们亟待解决。如果能够成功或者部分解决这类问题。我们可以很大程度上解决金融平台的风险问题
#### 1.3 数据集的大概情况是什么
我们从下载了数据集合[lending-club-loan-data.zip](https://s3.cn-north-1.amazonaws.com.cn/static-documents/nd101/AIPND/lending-club-loan-data.zip),其中loan.csv为主要数据集。

由于列数很多，我们无法对每个变量都进行分析，所以我们只对其中和金融相关的几个变量进行分析。

### 2.问题陈述
我们需要对数据进行处理，然后使用机器学习的方式进行数据拟合，来建议一个模型，使得在精度要求不高的情况下，有效的判断出在什么情况下的借贷风险是很高的。

### 3.评价指标
我们会将数据分为测试集还有验证集合，在测试集合的基础上对数据使用机器学习。然后将前者得到的模型运用到后者，将验证集合的结果与集合结果进行比对，计算出我们的模型的准确率。准确率，和可解释性才是我们的指标。

---
<a id="analysis"></a>
## Ⅱ.分析

### 1.数据的探索
我们需要将数据进行加工处理，主要包括以下操作：
1. 对我们感兴趣的列进行保留，不感兴趣的列进行删除
2. 对一些变量进行筛选，保留置信区间内的数据
3. 将数据分为测试集，验证集

### 2.探索性可视化
我们队一些数据进行可视化处理，分析出变量的分布，以及大概对结果的影响

### 3.算法和技术
对于只是区别是否的模型，我们只需要使用逻辑回归处理即可


---
<a id="implementation"></a>
## Ⅲ. 实现
在第三节中，你需要引入自己选定的模型，并使用数据集中的数据去训练兵优化它，使得最终的模型能够得到想要的结果

### 1.数据预处理
由于得到的变量比较多，我们不可能对所有变量都一一研究，为此我们选出了我们认为很大可能相关的变量。
下面是我们选择的有关变量：

| 名称 | 解释 |
| --- | --- |
| loan_amnt| 借贷总金额，按照常理，借贷越多，风险越大|
| term | 还款时间|
| installment，annual_inc| 每月还款与年薪，如果每月还款的金额占比较大，风险较高 |
| grade | 等级相关 |
| emp_length | 工作时间长，很可能信用就高 |
| home_ownership | 租来的房子会让风险增大 |
| verification_status | 验证后的风险会有变化 |
| pymnt_plan |  |
| purpose | 借贷目的对风险的影响 |
| application_type | 个人贷款，和有保证人的贷款风险不一样 |
| risk | 经过计算后的是否具有风险 |

### 2.执行过程
除了将我们关心的数据提取出来，我们还要对数据进行处理，主要包括如下几个方面：
1. 数值类型不用处理
2. 分类类型添加虚拟变量
3. 我们还要将数据分为训练集，验证集
### 3.完善

In [1]:
import pandas as pd
import statsmodels.api as sm
import data_trans
import data_sim
# 具体内容请参考脚本 
# data_sim.py 
# data_trans.py
# 提取我们需要的数据
data_trans.data_trans()
# 对数据进行拟合
data_sim.print_sim_result()

         Current function value: 0.480897
         Iterations: 35


/Users/dev/proj/python/lending-club-loan/venv/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/dev/proj/python/lending-club-loan/venv/lib/python3.7/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/Users/dev/proj/python/lending-club-loan/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/dev/proj/python/lending-club-loan/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/dev/proj/python/lending-club-loan/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal

                           Logit Regression Results                           
Dep. Variable:                   risk   No. Observations:               242697
Model:                          Logit   Df Residuals:                   242653
Method:                           MLE   Df Model:                           43
Date:                Fri, 02 Aug 2019   Pseudo R-squ.:                 0.07668
Time:                        23:34:19   Log-Likelihood:            -1.1671e+05
converged:                      False   LL-Null:                   -1.2640e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept             -0.5882        nan        nan        nan         nan         nan
loan_amnt          -1.964e-05   4.18e-06     -4.700      0.000   -2.78e-05   -1.14e-05
installment         

---
<a id="result"></a>
## IV. 结果
根据上述内容，我们得出一个还算是比较复杂的模型，由于分类变量衍生出来的虚拟变量较多，我们的模型稍微有点复杂。

### 模型的评价与验证
总体而言，我们还是得到了自己想要的模型，虽然稍微复杂一些。
而且我们还使用了测试集合，通过验证，我们的正确率达到了71，还是一个比较不错的结果

---
<a id="conclusion"></a>
## V. 项目结论

### 对项目的思考
从这个项目，我们清楚的了解了整个建模的过程。在整个过程中，我们发现在数据采样的时候遇到一些问题。
我们要尽量覆盖所有的数据，但是由于程序的限制，我们的数据并不是越多越好，如上，我们的数据导致计算的缓慢，甚至会提示达到数量的上限
由于是一个逻辑归一拟合，我们没有对置信区间做出处理。


### 需要作出的改进
1. 我们可以对数据进行进一步的处理，得到置信区间，或许对我们的结果有所帮助
2. 在计算正确率的时候，我们就以0.5为界限，其实我们还可以进一步优化，这个界限不一定是0.5
3. 如2中所述，如果我们再对界限取最优解，那么我们会将数据集合分为训练集，优化集，验证集

